<div style="background: linear-gradient(135deg, #034694 0%, #1E8449 50%, #D4AC0D 100%); color: white; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.2);">
    <h1 style="color: #FFF; text-shadow: 1px 1px 3px rgba(0,0,0,0.5);">🎓 Interactive Fine-Tuning Exercise</h1>
    <p style="font-size: 16px; line-height: 1.6;">
        Learn to fine-tune AI models using the Azure OpenAI SDK through hands-on practice. This interactive exercise will guide you through each step with questions, explanations, and code execution.
    </p>
</div>

<div style="background: #f8f9fa; border-left: 4px solid #0078d4; padding: 15px; margin: 10px 0; border-radius: 0 8px 8px 0;">
    <h3 style="color: #0078d4; margin-top: 0;">📋 Learning Objectives</h3>
    <ul style="margin-bottom: 0;">
        <li>Understand how to use the Azure OpenAI SDK for fine-tuning</li>
        <li>Learn the complete fine-tuning workflow step by step</li>
        <li>Practice working with training data and model configurations</li>
        <li>Monitor and evaluate fine-tuning jobs</li>
    </ul>
</div>

In [ ]:
# Progress Tracking System
import json
from datetime import datetime

class ProgressTracker:
    def __init__(self):
        self.total_steps = 8
        self.completed_steps = 0
        self.step_status = {}
        
    def complete_step(self, step_number, step_name):
        self.completed_steps += 1
        self.step_status[step_number] = {
            'name': step_name,
            'completed': True,
            'timestamp': datetime.now().strftime("%H:%M:%S")
        }
        self.show_progress()
        
    def show_progress(self):
        progress_percentage = (self.completed_steps / self.total_steps) * 100
        filled_blocks = int(progress_percentage // 10)
        empty_blocks = 10 - filled_blocks
        
        progress_bar = "█" * filled_blocks + "░" * empty_blocks
        
        print(f"\n{'='*60}")
        print(f"📊 PROGRESS: {self.completed_steps}/{self.total_steps} steps completed")
        print(f"[{progress_bar}] {progress_percentage:.1f}%")
        
        if self.completed_steps > 0:
            print(f"\n✅ Completed Steps:")
            for step_num, status in self.step_status.items():
                print(f"   Step {step_num}: {status['name']} - {status['timestamp']}")
                
        if self.completed_steps < self.total_steps:
            next_step = self.completed_steps + 1
            print(f"\n🎯 Next: Step {next_step}")
        else:
            print(f"\n🎉 All steps completed! Great job!")
        print(f"{'='*60}\n")

# Initialize progress tracker
progress = ProgressTracker()
progress.show_progress()

<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">📝 Step 1: Environment Setup & Azure SDK</h2>
    <p style="margin-bottom: 0;">Before we start fine-tuning, we need to set up our environment and understand the Azure OpenAI SDK.</p>
</div>

<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #856404; margin-top: 0;">🤔 Knowledge Check</h4>
    <p style="color: #856404; margin-bottom: 10px;"><strong>Question:</strong> What are the three essential environment variables needed to connect to Azure OpenAI using the SDK?</p>
    <details style="margin-top: 10px;">
        <summary style="cursor: pointer; color: #0078d4; font-weight: bold;">Click to reveal answer</summary>
        <div style="margin-top: 10px; padding: 10px; background: #d4edda; border-radius: 5px;">
            <strong>Answer:</strong> The three essential environment variables are:
            <ul>
                <li><code>AZURE_OPENAI_API_KEY</code> - Your API key for authentication</li>
                <li><code>AZURE_OPENAI_ENDPOINT</code> - The endpoint URL for your Azure OpenAI resource</li>
                <li><code>AZURE_OPENAI_API_VERSION</code> - The API version to use (optional, has defaults)</li>
            </ul>
        </div>
    </details>
</div>

In [ ]:
# Step 1: Environment Setup
import os

# Check environment variables
print("🔍 Checking environment variables...")

openai_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model_name = "gpt-4.1"
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-02-01-preview")

if not openai_key or not openai_endpoint:
    print("❌ Error: Missing AZURE_OPENAI_KEY or AZURE_OPENAI_ENDPOINT environment variable.")
    print("💡 Make sure to set these before proceeding!")
else:
    print("✅ Environment variables found!")

print(f"📋 Configuration:")
print(f"   Model: {model_name}")
print(f"   API Version: {api_version}")
print(f"   Endpoint: {'✅ Set' if openai_endpoint else '❌ Missing'}")
print(f"   API Key: {'✅ Set' if openai_key else '❌ Missing'}")

# Complete this step
progress.complete_step(1, "Environment Setup & Azure SDK")

<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">📂 Step 2: Data Preparation & Validation</h2>
    <p style="margin-bottom: 0;">Let's examine our training data and understand how it should be formatted for fine-tuning.</p>
</div>

<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #856404; margin-top: 0;">🤔 Knowledge Check</h4>
    <p style="color: #856404; margin-bottom: 10px;"><strong>Question:</strong> What format should training data be in for Azure OpenAI fine-tuning, and what is the required structure for each example?</p>
    <details style="margin-top: 10px;">
        <summary style="cursor: pointer; color: #0078d4; font-weight: bold;">Click to reveal answer</summary>
        <div style="margin-top: 10px; padding: 10px; background: #d4edda; border-radius: 5px;">
            <strong>Answer:</strong> Training data should be in JSONL format (JSON Lines), where each line contains:
            <ul>
                <li>A JSON object with a <code>"messages"</code> key</li>
                <li>The messages array contains conversation turns with <code>"role"</code> and <code>"content"</code></li>
                <li>Roles can be "system", "user", or "assistant"</li>
                <li>Example: <code>{"messages": [{"role": "user", "content": "Hello"}, {"role": "assistant", "content": "Hi there!"}]}</code></li>
            </ul>
        </div>
    </details>
</div>

In [ ]:
# Step 2: Data Preparation
import json

print("📂 Loading and validating training data...")

# Define data files
training_file = "data/basic_sft_training.jsonl" 
validation_file = "data/basic_sft_validation.jsonl"

try:
    # Load the training set
    with open(training_file, 'r', encoding='utf-8') as f:
        training_dataset = [json.loads(line) for line in f]

    print(f"✅ Training set loaded: {len(training_dataset)} examples")
    print(f"\n📋 First training example:")
    for i, message in enumerate(training_dataset[0]["messages"]):
        role_emoji = {"system": "🤖", "user": "👤", "assistant": "🔮"}
        emoji = role_emoji.get(message["role"], "💬")
        print(f"   {emoji} {message['role'].upper()}: {message['content'][:100]}...")

    # Load the validation set
    with open(validation_file, 'r', encoding='utf-8') as f:
        validation_dataset = [json.loads(line) for line in f]

    print(f"\n✅ Validation set loaded: {len(validation_dataset)} examples")
    
    print(f"\n📊 Data Summary:")
    print(f"   Training examples: {len(training_dataset)}")
    print(f"   Validation examples: {len(validation_dataset)}")
    print(f"   Total examples: {len(training_dataset) + len(validation_dataset)}")
    
    # Complete this step
    progress.complete_step(2, "Data Preparation & Validation")
    
except FileNotFoundError as e:
    print(f"❌ Error: Could not find data file - {e}")
    print("💡 Make sure the data files exist in the correct location!")

<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">🔢 Step 3: Token Analysis</h2>
    <p style="margin-bottom: 0;">Understanding token counts is crucial for fine-tuning costs and performance.</p>
</div>

<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #856404; margin-top: 0;">🤔 Knowledge Check</h4>
    <p style="color: #856404; margin-bottom: 10px;"><strong>Question:</strong> Why is it important to analyze token counts before fine-tuning? What encoding should you use for GPT-4 models?</p>
    <details style="margin-top: 10px;">
        <summary style="cursor: pointer; color: #0078d4; font-weight: bold;">Click to reveal answer</summary>
        <div style="margin-top: 10px; padding: 10px; background: #d4edda; border-radius: 5px;">
            <strong>Answer:</strong> Token analysis is important because:
            <ul>
                <li>Fine-tuning costs are based on the number of tokens processed</li>
                <li>Very long examples may be truncated or cause training issues</li>
                <li>Understanding token distribution helps optimize your dataset</li>
                <li>For GPT-4 models, use the <code>o200k_base</code> encoding</li>
            </ul>
        </div>
    </details>
</div>

In [ ]:
# Step 3: Token Analysis
import tiktoken
import numpy as np

print("🔢 Analyzing token counts...")

# Get the correct encoding for GPT-4 models
encoding = tiktoken.get_encoding("o200k_base")  # Default for GPT-4 models

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    """Calculate total tokens in a conversation."""
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # Every reply is primed with assistant
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    """Calculate tokens in assistant responses only."""
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    """Print statistical distribution of token counts."""
    print(f"\n📊 Distribution of {name}:")
    print(f"   Min/Max: {min(values):,} / {max(values):,}")
    print(f"   Mean/Median: {np.mean(values):.1f} / {np.median(values):.1f}")
    print(f"   5th/95th percentile: {np.quantile(values, 0.05):.1f} / {np.quantile(values, 0.95):.1f}")

# Analyze both datasets
files_data = [(training_file, training_dataset), (validation_file, validation_dataset)]

for file_name, dataset in files_data:
    print(f"\n📁 Analyzing: {file_name}")
    
    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens per example")
    print_distribution(assistant_tokens, "assistant tokens per example")
    print("-" * 50)

print("\n✅ Token analysis complete!")
print("💡 Look for examples that are unusually long - they might need review")

# Complete this step
progress.complete_step(3, "Token Analysis")

<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">☁️ Step 4: Azure OpenAI Client Setup</h2>
    <p style="margin-bottom: 0;">Now let's create the Azure OpenAI client to interact with the fine-tuning APIs.</p>
</div>

<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #856404; margin-top: 0;">🤔 Knowledge Check</h4>
    <p style="color: #856404; margin-bottom: 10px;"><strong>Question:</strong> What class from the OpenAI library do you use to create a client for Azure OpenAI, and what parameters does it require?</p>
    <details style="margin-top: 10px;">
        <summary style="cursor: pointer; color: #0078d4; font-weight: bold;">Click to reveal answer</summary>
        <div style="margin-top: 10px; padding: 10px; background: #d4edda; border-radius: 5px;">
            <strong>Answer:</strong> Use the <code>AzureOpenAI</code> class with these required parameters:
            <ul>
                <li><code>azure_endpoint</code> - Your Azure OpenAI endpoint URL</li>
                <li><code>api_key</code> - Your API key for authentication</li>
                <li><code>api_version</code> - The API version to use</li>
            </ul>
            <code>client = AzureOpenAI(azure_endpoint=endpoint, api_key=key, api_version=version)</code>
        </div>
    </details>
</div>

In [ ]:
# Step 4: Create Azure OpenAI Client
from openai import AzureOpenAI

print("☁️ Creating Azure OpenAI client...")

try:
    # Create the client with our environment variables
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2025-02-01-preview")
    )
    
    print("✅ Azure OpenAI client created successfully!")
    print(f"📋 Client configuration:")
    print(f"   Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
    print(f"   API Version: {os.getenv('AZURE_OPENAI_API_VERSION', '2025-02-01-preview')}")
    print(f"   Ready for fine-tuning operations: ✅")
    
    # Complete this step
    progress.complete_step(4, "Azure OpenAI Client Setup")
    
except Exception as e:
    print(f"❌ Error creating client: {e}")
    print("💡 Check your environment variables and try again!")

<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">📤 Step 5: Upload Training Data</h2>
    <p style="margin-bottom: 0;">Upload our training and validation files to Azure OpenAI for the fine-tuning process.</p>
</div>

<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #856404; margin-top: 0;">🤔 Knowledge Check</h4>
    <p style="color: #856404; margin-bottom: 10px;"><strong>Question:</strong> What method do you use to upload files to Azure OpenAI, and what purpose should you specify for fine-tuning files?</p>
    <details style="margin-top: 10px;">
        <summary style="cursor: pointer; color: #0078d4; font-weight: bold;">Click to reveal answer</summary>
        <div style="margin-top: 10px; padding: 10px; background: #d4edda; border-radius: 5px;">
            <strong>Answer:</strong> Use the <code>client.files.create()</code> method with:
            <ul>
                <li><code>file</code> parameter - the file object opened in binary mode</li>
                <li><code>purpose="fine-tune"</code> - specifies this file is for fine-tuning</li>
            </ul>
            <code>response = client.files.create(file=open("data.jsonl", "rb"), purpose="fine-tune")</code>
        </div>
    </details>
</div>

In [ ]:
# Step 5: Upload Training Data
print("📤 Uploading training data to Azure OpenAI...")

try:
    # Upload training file
    print("   ⏳ Uploading training file...")
    training_response = client.files.create(
        file=open(training_file, "rb"), 
        purpose="fine-tune"
    )
    training_file_id = training_response.id
    print(f"   ✅ Training file uploaded: {training_file_id}")

    # Upload validation file
    print("   ⏳ Uploading validation file...")
    validation_response = client.files.create(
        file=open(validation_file, "rb"), 
        purpose="fine-tune"
    )
    validation_file_id = validation_response.id
    print(f"   ✅ Validation file uploaded: {validation_file_id}")

    print(f"\n📋 File Upload Summary:")
    print(f"   Training file ID: {training_file_id}")
    print(f"   Validation file ID: {validation_file_id}")
    print(f"   Files ready for fine-tuning: ✅")
    
    print(f"\n💡 Tip: You can view these files in Azure AI Foundry Portal")
    print(f"   Go to your AI Project → Data Files tab to see uploaded files")
    
    # Complete this step
    progress.complete_step(5, "Upload Training Data")
    
except Exception as e:
    print(f"❌ Error uploading files: {e}")
    print("💡 Check file paths and permissions, then try again!")

<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">🚀 Step 6: Submit Fine-Tuning Job</h2>
    <p style="margin-bottom: 0;">Create and submit the fine-tuning job with our uploaded data and model configuration.</p>
</div>

<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #856404; margin-top: 0;">🤔 Knowledge Check</h4>
    <p style="color: #856404; margin-bottom: 10px;"><strong>Question:</strong> What method creates a fine-tuning job, and what are the required parameters? Why is the seed parameter useful?</p>
    <details style="margin-top: 10px;">
        <summary style="cursor: pointer; color: #0078d4; font-weight: bold;">Click to reveal answer</summary>
        <div style="margin-top: 10px; padding: 10px; background: #d4edda; border-radius: 5px;">
            <strong>Answer:</strong> Use <code>client.fine_tuning.jobs.create()</code> with:
            <ul>
                <li><code>training_file</code> - ID of uploaded training file</li>
                <li><code>model</code> - Base model to fine-tune (e.g., "gpt-4.1-2025-04-14")</li>
                <li><code>validation_file</code> - ID of validation file (optional)</li>
                <li><code>seed</code> - Controls reproducibility of training results</li>
            </ul>
            The seed parameter ensures you get consistent results when retraining with the same data.
        </div>
    </details>
</div>

In [ ]:
# Step 6: Submit Fine-Tuning Job
print("🚀 Submitting fine-tuning job...")

try:
    # Create the fine-tuning job
    response = client.fine_tuning.jobs.create(
        training_file=training_file_id,
        validation_file=validation_file_id,
        model="gpt-4.1-2025-04-14",  # Azure model name format (with dashes)
        seed=105  # For reproducible results
    )

    job_id = response.id
    
    print(f"✅ Fine-tuning job submitted successfully!")
    print(f"\n📋 Job Details:")
    print(f"   Job ID: {job_id}")
    print(f"   Status: {response.status}")
    print(f"   Model: {response.model}")
    print(f"   Training file: {response.training_file}")
    print(f"   Validation file: {response.validation_file}")
    
    print(f"\n⏰ The fine-tuning process will take some time to complete")
    print(f"   Typical duration: 10-60 minutes depending on data size")
    print(f"   You can monitor progress in the next step")
    
    # Store job_id for later steps
    globals()['job_id'] = job_id
    
    # Complete this step
    progress.complete_step(6, "Submit Fine-Tuning Job")
    
except Exception as e:
    print(f"❌ Error submitting job: {e}")
    print("💡 Check that your model is available in your region and try again!")

<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">📊 Step 7: Monitor Training Progress</h2>
    <p style="margin-bottom: 0;">Track the status of our fine-tuning job and monitor training progress.</p>
</div>

<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #856404; margin-top: 0;">🤔 Knowledge Check</h4>
    <p style="color: #856404; margin-bottom: 10px;"><strong>Question:</strong> How do you check the status of a fine-tuning job? What are the possible status values to expect?</p>
    <details style="margin-top: 10px;">
        <summary style="cursor: pointer; color: #0078d4; font-weight: bold;">Click to reveal answer</summary>
        <div style="margin-top: 10px; padding: 10px; background: #d4edda; border-radius: 5px;">
            <strong>Answer:</strong> Use <code>client.fine_tuning.jobs.retrieve(job_id)</code> to check status.
            <br><strong>Possible statuses:</strong>
            <ul>
                <li><code>"queued"</code> - Job is waiting to start</li>
                <li><code>"running"</code> - Training is in progress</li>
                <li><code>"succeeded"</code> - Training completed successfully</li>
                <li><code>"failed"</code> - Training failed with errors</li>
            </ul>
        </div>
    </details>
</div>

In [ ]:
# Step 7: Monitor Training Progress
import time
from datetime import datetime

print("📊 Monitoring fine-tuning job progress...")

def check_job_status():
    """Check and display current job status."""
    try:
        response = client.fine_tuning.jobs.retrieve(job_id)
        status = response.status
        
        # Status emojis for better visualization
        status_emojis = {
            "queued": "⏳",
            "running": "🔄", 
            "succeeded": "✅",
            "failed": "❌"
        }
        
        emoji = status_emojis.get(status, "📋")
        print(f"\n{emoji} Status: {status.upper()}")
        
        if hasattr(response, 'created_at'):
            created_time = datetime.fromtimestamp(response.created_at)
            print(f"   Created: {created_time.strftime('%H:%M:%S')}")
        
        if status == "running":
            print(f"   🔥 Training in progress...")
            print(f"   💡 This may take 10-60 minutes depending on data size")
        elif status == "succeeded":
            print(f"   🎉 Training completed successfully!")
            if hasattr(response, 'fine_tuned_model'):
                print(f"   🤖 Fine-tuned model: {response.fine_tuned_model}")
        elif status == "failed":
            print(f"   💥 Training failed")
            if hasattr(response, 'error'):
                print(f"   Error: {response.error}")
        
        return status, response
        
    except Exception as e:
        print(f"❌ Error checking status: {e}")
        return None, None

# Check current status
status, response = check_job_status()

if status:
    print(f"\n📋 Job monitoring setup complete!")
    print(f"   Job ID: {job_id}")
    print(f"   Current status: {status}")
    
    if status in ["queued", "running"]:
        print(f"\n💡 To continue monitoring, run the next cell periodically")
        print(f"   Or check progress in Azure AI Foundry Portal")
    
    # Complete this step
    progress.complete_step(7, "Monitor Training Progress")
    
    # Store the response for next step
    globals()['latest_response'] = response

<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); color: white; padding: 15px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">🎯 Step 8: Retrieve & Deploy Model</h2>
    <p style="margin-bottom: 0;">Once training is complete, retrieve the fine-tuned model information and prepare for deployment.</p>
</div>

<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #856404; margin-top: 0;">🤔 Knowledge Check</h4>
    <p style="color: #856404; margin-bottom: 10px;"><strong>Question:</strong> After a fine-tuning job succeeds, how do you get the fine-tuned model name? What other useful information can you retrieve about the training job?</p>
    <details style="margin-top: 10px;">
        <summary style="cursor: pointer; color: #0078d4; font-weight: bold;">Click to reveal answer</summary>
        <div style="margin-top: 10px; padding: 10px; background: #d4edda; border-radius: 5px;">
            <strong>Answer:</strong> After success, the job response contains:
            <ul>
                <li><code>response.fine_tuned_model</code> - The name of your fine-tuned model</li>
                <li><code>response.trained_tokens</code> - Total tokens used in training</li>
                <li>Use <code>client.fine_tuning.jobs.list_events(job_id)</code> for training logs</li>
                <li>Use <code>client.fine_tuning.jobs.checkpoints.list(job_id)</code> for checkpoints</li>
            </ul>
        </div>
    </details>
</div>

In [ ]:
# Step 8: Retrieve Fine-Tuned Model
print("🎯 Retrieving fine-tuned model information...")

try:
    # Get the latest job status
    response = client.fine_tuning.jobs.retrieve(job_id)
    
    print(f"📋 Final Job Status: {response.status}")
    
    if response.status == "succeeded":
        print(f"🎉 Fine-tuning completed successfully!")
        
        # Get the fine-tuned model name
        fine_tuned_model = response.fine_tuned_model
        print(f"🤖 Fine-tuned model: {fine_tuned_model}")
        
        # Display training summary
        if hasattr(response, 'trained_tokens'):
            print(f"📊 Training summary:")
            print(f"   Tokens processed: {response.trained_tokens:,}")
        
        # Get training events (last few)
        print(f"\n📜 Recent training events:")
        try:
            events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=3)
            for event in events.data:
                print(f"   • {event.message}")
        except:
            print("   Unable to retrieve events")
        
        print(f"\n🚀 Next steps:")
        print(f"   1. Deploy the model in Azure AI Foundry Portal")
        print(f"   2. Test it in the Chat Playground")
        print(f"   3. Compare performance with the base model")
        
        # Store model name for potential future use
        globals()['fine_tuned_model'] = fine_tuned_model
        
    elif response.status == "failed":
        print(f"❌ Fine-tuning failed")
        print(f"💡 Check the error details and training data")
        
    else:
        print(f"⏳ Training still in progress: {response.status}")
        print(f"💡 Wait for completion before proceeding")
    
    # Complete this step regardless of status
    progress.complete_step(8, "Retrieve & Deploy Model")
    
    print(f"\n🎓 Congratulations! You've completed the fine-tuning exercise!")
    
except Exception as e:
    print(f"❌ Error retrieving model: {e}")
    print("💡 Make sure the job exists and try again!")

<div style="background: linear-gradient(135deg, #228B22 0%, #32CD32 100%); color: white; padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: #fff;">🎉 Exercise Complete!</h2>
    <p style="margin-bottom: 0;">Great job! You've successfully learned how to use the Azure OpenAI SDK for fine-tuning. You now understand the complete workflow from data preparation to model deployment.</p>
</div>

<div style="background: #e8f5e8; border: 1px solid #28a745; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #155724; margin-top: 0;">🏆 What You've Learned</h4>
    <ul style="color: #155724;">
        <li>✅ How to set up the Azure OpenAI SDK environment</li>
        <li>✅ Preparing and validating training data for fine-tuning</li>
        <li>✅ Analyzing token counts to optimize costs and performance</li>
        <li>✅ Uploading training files using the Azure OpenAI SDK</li>
        <li>✅ Creating and submitting fine-tuning jobs programmatically</li>
        <li>✅ Monitoring training progress and job status</li>
        <li>✅ Retrieving fine-tuned models and understanding deployment steps</li>
        <li>✅ Best practices for the complete fine-tuning workflow</li>
    </ul>
</div>

<div style="background: #f8f9fa; border: 1px solid #dee2e6; padding: 15px; border-radius: 8px; margin: 10px 0;">
    <h4 style="color: #495057; margin-top: 0;">🔗 Additional Resources</h4>
    <ul style="color: #495057;">
        <li><a href="https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/fine-tuning" target="_blank">Azure OpenAI Fine-tuning Documentation</a></li>
        <li><a href="https://platform.openai.com/docs/guides/fine-tuning" target="_blank">OpenAI Fine-tuning Guide</a></li>
        <li><a href="https://learn.microsoft.com/en-us/azure/ai-foundry/" target="_blank">Azure AI Foundry Documentation</a></li>
        <li><a href="https://github.com/Azure/azure-openai-samples" target="_blank">Azure OpenAI Code Samples</a></li>
    </ul>
</div>

In [ ]:
# Final progress summary
print("🎓 Exercise Summary")
print("="*50)
progress.show_progress()

if progress.completed_steps == progress.total_steps:
    print("🏆 CONGRATULATIONS! 🏆")
    print("You've mastered the Azure OpenAI SDK for fine-tuning!")
    print("Ready to build amazing AI applications! 🚀")
else:
    print(f"📚 Continue learning! {progress.total_steps - progress.completed_steps} steps remaining.")